In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv(dotenv_path="app/resources/.env")

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key = "groqclod_API_key"
    # other params...
)

response = llm.invoke("The first country to reach mars with low budget")
print(response.content)

While no country has yet successfully landed a mission on Mars with a low budget, there are a few contenders that have made notable attempts. Here are a few examples:

1. **India's Mangalyaan (Mars Orbiter Mission)**: Launched in 2013, Mangalyaan was a low-cost mission that successfully entered Mars' orbit in 2014. The mission was developed by the Indian Space Research Organisation (ISRO) with a budget of approximately $74 million, which is relatively low compared to other Mars missions.
2. **China's Yinghuo-1**: Launched in 2011, Yinghuo-1 was a Chinese Mars orbiter that was designed to study the Martian atmosphere and geology. Although the mission failed to enter Mars' orbit due to a launch vehicle malfunction, it was notable for its low budget of approximately $100 million.
3. **United Arab Emirates' Hope Probe**: Launched in 2020, the Hope Probe is a Mars orbiter developed by the Mohammed bin Rashid Space Centre (MBRSC) in the United Arab Emirates. The mission was designed to study

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1/")
page_data = loader.load().pop().page_content

print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.















Llama 3.1 | Model Cards and Prompt formats





























Skip to main contentDocumentationTrust & SafetyCommunityTry 405B on Meta AIDownload models

















DocumentationOverviewModels Getting the Models Running Llama How-To Guides Integration Guides Community Support CommunityCommunity StoriesOpen Innovation AI Research CommunityLlama Impact GrantsResourcesAI at Meta BlogMeta NewsroomFAQPrivacy PolicyTermsCookiesTrust & SafetyOverviewResponsible Use GuideDocumentationOverviewModels Getting the Models Running Llama How-To Guides Integration Guides Community Support CommunityCommunity StoriesOpen Innovation AI Research CommunityLlama Impact GrantsResourcesAI at Meta BlogMeta NewsroomFAQPrivacy PolicyTermsCookiesTrust & SafetyOverviewResponsible Use GuideDocumentationOverviewModels Getting the Models Running Llama How-To Guides Integration Guides Community Support CommunityCommunity StoriesOpen Innovation AI Research CommunityLlama Impact GrantsResourc

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm

res = chain_extract.invoke(input = {'page_data':page_data})
type(res.content)
print(res.content)

[
  {
    "role": "Software Engineer",
    "experience": "3-5 years",
    "skills": "Python, Java, C++, AWS, Azure",
    "description": "Design, develop, and test software applications"
  },
  {
    "role": "Data Scientist",
    "experience": "5-7 years",
    "skills": "Python, R, SQL, Machine Learning, Data Visualization",
    "description": "Analyze and interpret complex data to inform business decisions"
  },
  {
    "role": "DevOps Engineer",
    "experience": "3-5 years",
    "skills": "AWS, Azure, Docker, Kubernetes, Jenkins",
    "description": "Ensure the smooth operation of software systems and infrastructure"
  }
]


In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer',
  'experience': '3-5 years',
  'skills': 'Python, Java, C++, AWS, Azure',
  'description': 'Design, develop, and test software applications'},
 {'role': 'Data Scientist',
  'experience': '5-7 years',
  'skills': 'Python, R, SQL, Machine Learning, Data Visualization',
  'description': 'Analyze and interpret complex data to inform business decisions'},
 {'role': 'DevOps Engineer',
  'experience': '3-5 years',
  'skills': 'AWS, Azure, Docker, Kubernetes, Jenkins',
  'description': 'Ensure the smooth operation of software systems and infrastructure'}]

In [5]:
type(json_res)

list

In [6]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [7]:
import uuid
import chromadb

client = chromadb.PersistentClient('VectorStore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents = row["Techstack"],
                       metadatas = {"links":row["Links"]},
                       ids=[str(uuid.uuid4())]
                       )

In [8]:
links = collection.query(query_texts=["Experience in Python","Expertise in React","Expertise in Machine Learning"],n_results=2).get('metadatas',[])
links


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [9]:
for j in job:
    print(f"Role: {j['role']}, Skills: {j['skills']}")


NameError: name 'job' is not defined

In [28]:
prompt_email = PromptTemplate.from_template(
    """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
    
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description":str(job),"link_list":links})
print(res.content)


Subject: Expert Software Solutions for Your Business Needs

Dear [Client's Name],

I hope this email finds you well. As a Business Development Executive at AtliQ, I came across your job postings for a Software Engineer, Data Scientist, and DevOps Engineer. I am excited to introduce you to our team of experts who can help you achieve your business goals through cutting-edge software solutions.

At AtliQ, we specialize in AI and software consulting, empowering businesses like yours to streamline processes, enhance scalability, and reduce costs. Our team of seasoned professionals has a proven track record of delivering tailored solutions that meet the unique needs of our clients.

Our capabilities align perfectly with your job requirements. We have expertise in Python, Java, C++, AWS, Azure, and other technologies that your job postings mention. Our team can design, develop, and test software applications that meet your business needs. We can also analyze and interpret complex data to inf